# Utilisation d'Agents C# (Semantic Kernel) depuis Python via pythonnet

Ce **troisième notebook** illustre comment, en Python, on peut :

1. **Installer et configurer** `pythonnet`  
2. **Charger** la DLL .NET (compilée en C#) qui contient notre code d’agent (`AutoInvokeSKAgentsNotebookUpdater` et consorts)  
3. **Instancier** et **appeler** ces classes depuis Python  

**Prérequis** :  
- Avoir le fichier `MyIA.AI.Notebooks.dll` déjà compilé (typiquement dans `..\..\bin\Debug\net9.0\MyIA.AI.Notebooks.dll`).  
- Disposer d’un environnement Python (3.x) qui peut installer `pythonnet`.  



In [ ]:
# ============================
# Bloc 1 : Installation pythonnet & configuration
# ============================

# Installe pythonnet. À n’exécuter qu’une seule fois (enlevez --quiet si besoin)
%pip install pythonnet

# On peut vérifier la version
import pkg_resources
print("pythonnet version :", pkg_resources.get_distribution("pythonnet").version)

print("Installation pythonnet : OK")


## Bloc 2 : Importation de la DLL .NET

Nous allons :  
1. **Modifier** le chemin Python (`sys.path`) pour inclure le dossier où se trouve la DLL .NET.  
2. **Charger** la DLL (`clr.AddReference`)  
3. **Importer** les namespaces/classes C# (ex. `MyIA.AI.Notebooks`).

Note : Adaptez `dll_path` si nécessaire.


In [ ]:
# ============================
# Bloc 2 : Charger la DLL .NET dans Python
# ============================

import sys
import clr

# <-- Ajustez ce chemin vers le dossier contenant MyIA.AI.Notebooks.dll
dll_path = r"..\..\bin\Debug\net9.0"

# On ajoute ce dossier dans sys.path
if dll_path not in sys.path:
    sys.path.append(dll_path)

# On charge la DLL (sans l'extension .dll)
clr.AddReference("MyIA.AI.Notebooks")

print("Référence à 'MyIA.AI.Notebooks.dll' ajoutée avec succès.")


# Vérifier le contenu chargé par `clr.AddReference` dans pythonnet

Pour **examiner** ce qui a été effectivement chargé en mémoire via `clr.AddReference`, vous pouvez :

1. **Lister les assemblées (.NET) actuellement référencées**  
2. **Inspecter** les types (classes, etc.) exportés par chaque assembly




In [ ]:
import clr
from System import AppDomain

for asm in AppDomain.CurrentDomain.GetAssemblies():
    if asm.GetName().Name == "MyIA.AI.Notebooks":
        print(f"\nAssembly: {asm.GetName().Name}")
        try:
            for t in asm.GetExportedTypes():
                print("  ->", t.FullName)
        except:
            pass


## Bloc 3 : Import des classes C# du namespace `MyIA.AI.Notebooks`

On suppose que les classes suivantes sont exposées :  
- **`AutoInvokeSKAgentsNotebookUpdater`**  
- **`DisplayLogger`** (et son provider)  
- etc.

Ensuite, on pourra instancier et exécuter du code C# directement depuis Python.


In [ ]:
# ============================
# Bloc 3 : Import des classes du namespace
# ============================

# On importe le namespace racine
from MyNotebookLib import AutoInvokeSKAgentsNotebookUpdater



# S'il y a d'autres classes à importer, faites:
# from MyIA.AI.Notebooks import DisplayLogger, DisplayLoggerProvider, NotebookUpdaterBase, ...

print("Import des classes C# terminé.")


## Bloc 4 : Exemple d'utilisation

- On va créer une instance de `AutoInvokeSKAgentsNotebookUpdater`.
- On va définir un logger factice ou inexistant (selon la configuration).
- On va lancer la méthode `UpdateNotebookAsync()` pour lancer l'agent.

Note : Comme la méthode est asynchrone en C#, on l'appelle via `await` dans un contexte `asyncio` Python.


In [ ]:
# ============================
# Bloc 4 : Exemple d'appel d'agent
# ============================
import nest_asyncio
nest_asyncio.apply()

import asyncio



# Facultatif si vous avez un logger C# 
# from Microsoft.Extensions.Logging import LogLevel
# from MyIA.AI.Notebooks import DisplayLogger

async def run_agent_example():
    # logger = DisplayLogger("PyNotebookUpdater", LogLevel.Debug)  # Si vous avez un logger
    logger = None  # ou passer None si le constructeur l'accepte

    # Chemin où sera généré le notebook "cible" à manipuler
    notebook_path = r".\Workbook-Template-Python.ipynb"

    # On crée l'updater
    updater = AutoInvokeSKAgentsNotebookUpdater(notebook_path, logger)

    # On définit un template ou une instruction initiale
    # (Équivalent de: autoInvokeUpdater.SetStartingNotebookFromTemplate(...))
    # La méthode correspondante est "SetStartingNotebookFromTemplate" :
    updater.SetStartingNotebookFromTemplate("""
Créer un notebook Python, capable de requêter DBpedia via SPARQL, 
puis tracer un graphique Plotly. 
Ensuite, 
1) corriger d'éventuels bugs 
2) valider la sortie
""")

    # On exécute l'agent => C# : await autoInvokeUpdater.UpdateNotebookAsync()
    await updater.UpdateNotebookAsync()

    print("Agent SK terminé avec succès !")


# Lancement
asyncio.run(run_agent_example())


## Explications

1. **`NotebookPath`** : Le fichier `.ipynb` que vous souhaitez générer / mettre à jour.
2. **`updater.SetStartingNotebookFromTemplate(...)`** : Injecte la consigne ou le template initial dans le notebook cible.
3. **`await updater.UpdateNotebookAsync()`** : Lance l'agent C# (AutoInvokeSKAgentsNotebookUpdater) qui va orchestrer Semantic Kernel pour incrémenter / améliorer le notebook.

Vous pouvez ensuite rouvrir le fichier `.ipynb` généré (dans un Jupyter Lab/VS Code) pour voir le résultat.


## Conclusion

Grâce à `pythonnet`, on peut :
- **Charger** des DLL C# (compilées .NET).
- **Importer** leurs classes.
- **Instancier** et **appeler** leurs méthodes asynchrones ou synchrones, y compris du code Semantic Kernel.
- **Exploiter** des agents SK pour générer ou manipuler des notebooks interactifs, directement depuis un script/notebook Python.

Cela permet de **mélanger** l’écosystème Python avec des composants .NET plus sophistiqués, comme vos agents d’orchestration.
